In [1]:
# The function to get the reverse complementary sequences
rc = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
def revcomp(tmp_seq):
    return ''.join([rc[x] for x in tmp_seq[::-1]])

In [2]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [3]:
# Read primers
N1,N2,N3,RP={},{},{},{}
prim=(N1,N2,N3,RP)
filename_primers = '../data/2019nCoV_primers.fa'
primer_list = read_fasta(filename_primers)
for p_h, p_seq in primer_list.items():
    print(p_h, p_seq)
    if p_h.find('N1')!=-1:
        N1[p_h[-1]]=p_seq
    elif p_h.find('N2')!=-1:
        N2[p_h[-1]]=p_seq
    elif p_h.find('N3')!=-1:
        N3[p_h[-1]]=p_seq
    else:
        RP[p_h[-1]]=p_seq

2019-nCoV_N1-F GACCCCAAAATCAGCGAAAT
2019-nCoV_N1-R TCTGGTTACTGCCAGTTGAATCTG
2019-nCoV_N1-P ACCCCGCATTACGTTTGGTGGACC
2019-nCoV_N2-F TTACAAACATTGGCCGCAAA
2019-nCoV_N2-R GCGCGACATTCCGAAGAA
2019-nCoV_N2-P ACAATTTGCCCCCAGCGCTTCAG
2019-nCoV_N3-F GGGAGCCTTGAATACACCAAAA
2019-nCoV_N3-R TGTAGCACGATTGCAGCATTG
2019-nCoV_N3-P AYCACATTGGCACCCGCAATCCTG
RP-F AGATTTGGACCTGCGAGCG
RP-R GAGCGGCTGTCTCCACAAGT
RP-P TTCTGACCTGAAGGCTCTGCGCG


In [4]:
# Read genomes
filename_genomes = '../data/2019nCoV_genomes.2020_02_03.fa'
genome_list = read_fasta(filename_genomes)
for g_h, g_seq in genome_list.items():
    print(g_h.split()[0], g_seq[:25])

ENA|MN988668|MN988668.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN988669|MN988669.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN938384|MN938384.1 CAACCAACTTTCGATCTCTTGTAGA
ENA|MN975262|MN975262.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996527|MN996527.1 AACCAACTTTCGATCTCTTGTAGAT
ENA|MN996528|MN996528.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996529|MN996529.1 TACCTTCCCAGGTAACAAACCAACC
ENA|MN996530|MN996530.1 CCTTCCCAGGTAACAAACCAACCAA
ENA|MN996531|MN996531.1 ACCTTCCCAGGTAACAAACCAACCA
ENA|MN908947|MN908947.3 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN985325|MN985325.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN988713|MN988713.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN994467|MN994467.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN994468|MN994468.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN997409|MN997409.1 ATTAAAGGTTTATACCTTCCCAGGT


In [5]:
# Q1. Find primers in the genome
# What is the length of amplicons generated by primers? Are they all same among the genomes?
primers=['N1','N2','N3','RP']
cnt=0
TF=True
for i in prim: #N1,N2,N3,RP
    amp_list=[]
    num=0
    for j in genome_list: #genome 1~15
        if(i['F'] in genome_list[j] and revcomp(i['R']) in genome_list[j]):
            F,revR=genome_list[j].find(i['F']),genome_list[j].find(revcomp(i['R']))
            amp=genome_list[j][F:revR+len(i['R'])]
            amp_list.extend(amp)
            if num!=0:
                if temp_amp!=amp:
                    TF=False
            temp_amp=amp
            num+=1
    if num!=0:
        print(primers[cnt], "-", len(amp_list)/num, amp)
        if TF==True:
            print("They are all same among the genomes")
        else:
            print("They are NOT all same among the genomes")
    else:
        print(primers[cnt], "- No amplicons")
    cnt+=1

N1 - 72.0 GACCCCAAAATCAGCGAAATGCACCCCGCATTACGTTTGGTGGACCCTCAGATTCAACTGGCAGTAACCAGA
They are all same among the genomes
N2 - 67.0 TTACAAACATTGGCCGCAAATTGCACAATTTGCCCCCAGCGCTTCAGCGTTCTTCGGAATGTCGCGC
They are all same among the genomes
N3 - 72.0 GGGAGCCTTGAATACACCAAAAGATCACATTGGCACCCGCAATCCTGCTAACAATGCTGCAATCGTGCTACA
They are all same among the genomes
RP - No amplicons


In [6]:
# Q2. Compare the number of amplicons and the number of genomes
# Can these primers detect all SARS-CoV-2 genomes?
primers=['N1','N2','N3','RP']
cnt,same=0,0
for i in prim: #N1,N2,N3,RP
    for j in genome_list: #genome 1~15
        if(i['F'] in genome_list[j] and revcomp(i['R']) in genome_list[j]):
            same+=1
    if same==len(genome_list):
        print(primers[cnt], "primers can detect all SARS-CoV-2 genomes")
    else:
        print(primers[cnt], "primers can NOT detect all SARS-CoV-2 gnomes")
    same=0
    cnt+=1

N1 primers can detect all SARS-CoV-2 genomes
N2 primers can detect all SARS-CoV-2 genomes
N3 primers can detect all SARS-CoV-2 genomes
RP primers can NOT detect all SARS-CoV-2 gnomes


In [7]:
# Q3. First, Read MERS and SARS genome fasta file like 2019nCoV
# Read MERS, SARS
filename_MERS = '../data/MERS_genomes.2020_02_03.fa'
filename_SARS = '../data/SARS_genomes.2020_02_03.fa'
MERS_list = read_fasta(filename_MERS)
for m_h, m_seq in genome_list.items():
    print(m_h.split()[0], m_seq[:25])
print('\n')
SARS_list = read_fasta(filename_SARS)
for s_h, s_seq in genome_list.items():
    print(s_h.split()[0], s_seq[:25])

ENA|MN988668|MN988668.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN988669|MN988669.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN938384|MN938384.1 CAACCAACTTTCGATCTCTTGTAGA
ENA|MN975262|MN975262.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996527|MN996527.1 AACCAACTTTCGATCTCTTGTAGAT
ENA|MN996528|MN996528.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996529|MN996529.1 TACCTTCCCAGGTAACAAACCAACC
ENA|MN996530|MN996530.1 CCTTCCCAGGTAACAAACCAACCAA
ENA|MN996531|MN996531.1 ACCTTCCCAGGTAACAAACCAACCA
ENA|MN908947|MN908947.3 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN985325|MN985325.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN988713|MN988713.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN994467|MN994467.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN994468|MN994468.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN997409|MN997409.1 ATTAAAGGTTTATACCTTCCCAGGT


ENA|MN988668|MN988668.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN988669|MN988669.1 TTAAAGGTTTATACCTTCCCAGGTA
ENA|MN938384|MN938384.1 CAACCAACTTTCGATCTCTTGTAGA
ENA|MN975262|MN975262.1 ATTAAAGGTTTATACCTTCCCAGGT
ENA|MN996527|MN996527.1 AACCAACTTTCGATCTCTTGTAGA

In [8]:
# Q3. Proceed as in Q2
# 3. Can these primers detect MERS genomes? How about SARS genomes?
primers=['N1','N2','N3','RP']
cnt=0
MTF,STF=False,False
for i in prim: #N1,N2,N3,RP
    for j in MERS_list:
        if(i['F'] in MERS_list[j] and revcomp(i['R']) in MERS_list[j]):
            MTF=True
    for k in SARS_list:
        if(i['F'] in SARS_list[k] and revcomp(i['R']) in SARS_list[k]):
            STF=True
    if(MTF and STF):
        print(primers[cnt], "can detect MERS and SARS genomes")
    elif MTF:
        print(primers[cnt], "can ONLY detect MERS genomes")
    elif STF:
        print(primers[cnt], "can ONLY detect SARS genomes")
    else:
        print(primers[cnt], "can NOT detect any MERS or SARS genomes")
    cnt+=1

N1 can NOT detect any MERS or SARS genomes
N2 can NOT detect any MERS or SARS genomes
N3 can NOT detect any MERS or SARS genomes
RP can NOT detect any MERS or SARS genomes


In [9]:
# For the Q4 and Q5, Extract those amplicon sequences
amp_list={}
primers=['N1','N2','N3','RP']
cnt=0
for i in prim: #N1,N2,N3,RP
    for j in genome_list: #genome 1~15
        if(i['F'] in genome_list[j] and revcomp(i['R']) in genome_list[j]):
            F,revR=genome_list[j].find(i['F']),genome_list[j].find(revcomp(i['R']))
            amp=genome_list[j][F:revR+len(i['R'])]
            amp_list["Amplicon generated by "+primers[cnt]+" in genome "+j]=amp
    cnt+=1
    
ofile = open("../data/amplicon_sequences.fa","w")
for a_h, a_seq in amp_list.items():
    ofile.write(">"+a_h+"\n"+a_seq+"\n")
ofile.close()

# makeblastdb -dbtype nucl -in 2019nCoV_genomes.2020_03_07.fa -out 2019nCoV_genomes.2020_03_07.blastdb

In [10]:
# Q4. blastn -db 2019nCoV_genomes.2020_03_07.blastdb -query amplicon_sequences.fa -outfmt 6 -out Q4.out
# 4. Run the BLAST to all SARS-CoV-2 genomes. Are they all perfectly matched? Is there any mismatch or gap?
f=open('../data/Q4.out', 'r')
TF,TFTF=True,False
for i in f:
    if i.split()[2]!='100.000':
        TF=False
        print("It has "+i.split()[4]+"mismatch and "+i.split()[5]+"gap in"+i.split()[1])
    if TF:
        TFTF=True
if TF:
    print("They all perfectly matched well :)")
elif TFTF:
    print("The other rest perfectly matched well :P")
else:
    print("Nothing matched :(")

They all perfectly matched well :)


In [11]:
# Q5. blastx -db 2019_nCoV_proteins.2020_03_27.blastdb -query amplicon_sequences.fa -outfmt 6 -out Q5.out
# 5. Run the BLAST to all SARS-CoV-2 proteomes. Are they all perfectly matched?
f=open('../data/Q5.out', 'r')
Pass,Fail=0,0
TF,TFTF=True,False
for i in f:
    if i.split()[2]!='100.000':
        TF=False
        Fail+=1
    else:
        Pass+=1
    if TF:
        TFTF=True

if TF:
    print("They all perfectly matched well :)")
elif TFTF:
    print("Only "+str('%.3f'%(Pass/(Pass+Fail)*100))+"% of the total proteome matched perfectly :P")
else:
    print("Nothing matched :(")

Only 75.712% of the total proteome matched perfectly :P
